# Total Sales Pipeline
Raw layer from "Venda Total" pipeline

In [18]:
from bs4 import BeautifulSoup
from io import BytesIO
from google.cloud import bigquery
import re
import os
import requests
import zipfile
import pandas as pd


In [ ]:
"""
Constants for the ANP Fuel Price Scraper
"""

BASE_URL = "https://www.gov.br/anp/pt-br/centrais-de-conteudo/paineis-dinamicos-da-anp/paineis-dinamicos-do-abastecimento/painel-dinamico-do-mercado-brasileiro-de-combustiveis-liquidos"

In [20]:
"""
ANP Fuel Price Scraper
This script scrapes the ANP website for fuel prices and uploads the data to
Google Cloud Storage.
"""

response = requests.get(BASE_URL, verify=False)
if response.status_code == 200:
    page = BeautifulSoup(response.content, "html.parser")
else:
    raise Exception("Failed to retrieve data from the URL.")

pattern = re.compile(r"Clique\s+aqu.*Líquidos", re.IGNORECASE)
a_tag = page.find('a', string=pattern)
panel = None
if a_tag and 'href' in a_tag.attrs:
    panel_link = a_tag['href']
    panel_text = a_tag.get_text(strip=True)
    panel = {'text': panel_text, 'link': panel_link}
else:
    panel = None

data = None
data_link = page.find('a', string='Veja também a base de dados utilizada no painel')
if data_link and 'href' in data_link.attrs:
    link = data_link['href']
    text = data_link.get_text(strip=True)
    updated_data = data_link.next_sibling
    if updated_data:
        li_text = updated_data.get_text(strip=True).split("em ")[1][0:-1].strip()
    else:
        li_text = "Data não disponível"
    data = {'text': text, 'link': link, 'updated_date': li_text}
else:
    data = None

if data and data.get('link'):
    download_link = data.get('link')

    gcs_base_path = "general_sales"
    final_path = "/general_sales.zip"
    gcs_path = f"/{gcs_base_path}{final_path}"

    response = requests.get(download_link, verify=False)
    response.raise_for_status()
    zip_bytes = BytesIO(response.content)


/home/fnascime/labs_vibra/pipeline_extracts/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/fnascime/labs_vibra/pipeline_extracts/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
"""
Extracts the general sales data from a zip file in Google Cloud Storage,
"""

with zipfile.ZipFile(zip_bytes) as zf:
	for file_info in zf.infolist():
		with zf.open(file_info) as file:
			file_name = file_info.filename
			if file_info.filename == "Liquidos_Vendas_Atual.csv":
				df = pd.read_csv(file, sep=";", encoding="latin1")
				break


df = df.rename(columns={
	"Ano": "ano",
	"Mês": "mes",
	"Agente Regulado": "agente_regulado",
	"Código do Produto": "codigo_do_produto",
	"Nome do Produto": "nome_do_produto",
	"Descrição do Produto": "descricao_do_produto",
	"Região Origem": "regiao_origem",
	"UF Origem": "uf_origem",
	"Região Destinatário": "regiao_destinatario",
	"UF Destino": "uf_destino",
	"Mercado Destinatário": "mercado_destinatario",
	"Quantidade de Produto (mil m³)": "quantidade_de_produto_mil_m3",
})

In [22]:
"""
    this code removes unused years from the dataframe,
    filters the dataframe by start and end date,
    drops the year and product_code columns,
    converts the year, month, and day into a date column,
    get all datas to insert into BigQuery
    iterate over the dataframe and append the data to the corresponding month datafram
    create a hashmap to store the dataframes for each month
"""

df['data_venda'] = pd.to_datetime(dict(year=df['ano'], month=df['mes'], day=1)).astype('datetime64[us]')
df = df.drop(columns=['mes', 'ano'])

df['data_venda'] = pd.to_datetime(df['data_venda']).dt.date

df['data_criacao'] = pd.Timestamp.now(tz='America/Sao_Paulo')
df['quantidade_de_produto_mil_m3'] = df['quantidade_de_produto_mil_m3'].astype(str).str.replace(',', '.').astype('float64')

In [ ]:
"""
Inserting data into BigQuery
"""

bq_client = bigquery.Client()
project_id = os.getenv("GOOGLE_CLOUD_PROJECT")
bq_dataset = "rw_ext_anp"
table_name = "venda_total"
table_id = f"{project_id}.{bq_dataset}.{table_name}"

job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND,
)

try:
    job = bq_client.load_table_from_dataframe(
        df, table_id, job_config=job_config
    )
    job.result()
    print(f"  Data for {table_id} inserted successfully.")
except Exception as e:
    print(f"  Error inserting data for {table_id}: {str(e)}")

print("Data insertion completed!")

/home/fnascime/labs_vibra/pipeline_extracts/.venv/lib/python3.12/site-packages/google/cloud/bigquery/_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


  Data for labs-vibra-final.rw_ext_anp.venda_total inserted successfully.
Data insertion completed!
